# Step 1: Import Libraries and Load the Dataset

In [ ]:
# Imports

#pathlib: modern, clean way to handle file and folder paths
from pathlib import Path

#shutil: used to copy files from raw folders to the train and test folders
import shutil

# nump useful for numerical operations
import numpy as np

import pandas as pd

#matplotlib.pyplot: uswed for visualizations
import matplotlib.pyplot as plt

#tensorflow and keras for deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
